In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import config
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Tutorial: Hyperparameter Search

In [3]:
import torch
import pytorch_lightning as pl
import optuna
from optuna.integration import PyTorchLightningPruningCallback

from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule
#from classification.models.MelSpectrogramCNN_8K import MelSpectrogramCNN_8KPLModule
from classification.models.DeepRecursiveCNN8k import DeepRecursiveCNN8kPLModule

from classification.models.SpectrogramCNN_8K_Dataset2 import SpectrogramCNN_8K_Dataset2_PLModule
from classification.models.CRNN_8k_D2 import CRNN8k_D2_PLModule
from classification.trainer.HyperParamSearch import MetricsCallback, save_model

from classification.models.AblationModel import AblationModelPLModule

In [4]:
from pytorch_lightning.callbacks import Callback

class SaveBestCallback(Callback): 
    def __init__(self, model_name = "newest_model", add_v_number = True):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None
        self.add_v_number = add_v_number

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            #save_path = self.model_name + (( "_v{}".format(trainer.logger.version) +  "_best.p") if self.add_v_number else "")
            save_path = self.model_name + str(pl_module.val_results_history[-1]["val_acc"]) + ".p"
            pl_module.save(save_path, overwrite_if_exists=True)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveBestCallback("optuna_ablationNormal_", add_v_number = False)

### Objective

In [5]:
def objective(trial):
    metrics_callback = MetricsCallback()  

    # here we sample the hyper params, similar as in our old random search
    trial_hparams = {"batch_size": 32,
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-1),
                     "p_dropout": trial.suggest_float("p_drop", 0, 1),
                     "lr_decay": trial.suggest_float("lr_decay", 0.7, 1),
                     "n_hidden": trial.suggest_int("n_hidden", 10, 1000),
                     #"lstm_hidden_size": trial.suggest_int("lstm_hidden_size", 10, 1000),
                     "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1),
                     "normal_spectrogram": True
                    }    

    model = AblationModelPLModule(trial_hparams)
    
    # create a trainer
    trainer = pl.Trainer(
        logger=pl.loggers.TensorBoardLogger(config.LOG_DIR, name=type(model.model).__name__),
        max_epochs=20,                                                               
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, cb],                                                   # save latest accuracy
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="validation_acc"), # early stopping
    )
    
    datasetHandler.load(model, 'training')#, dataset_id=config.DATASET_CONTROL)
    datasetHandler.load(model, 'validation')#,  dataset_id=config.DATASET_CONTROL)
    trainer.fit(model)

    # save model
    model.save("saved_modelsq/"+'{}.p'.format(trial.number))

    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["validation_acc"]

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner, study_name='ablationNormal_study', storage='sqlite:///ablationNormal_study.db', load_if_exists=True)

[I 2020-07-20 14:19:08,546] Using an existing study with name 'ablationNormal_study' instead of creating a new one.


In [6]:
#!mkdir saved_modelsq

### Run Search

In [ ]:
study.optimize(objective, n_trials=2000, timeout=1.5*21600) #9h

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.

   | Name           | Type          | Params
---------------------------------------------
0  | model          | AblationModel | 232 K 
1  | model.convs    | Sequential    | 162 K 
2  | model.convs.0  | BatchNorm2d   | 2     
3  | model.convs.1  | Conv2d        | 2 K   
4  | model.convs.2  | BatchNorm2d   | 40    
5  | model.convs.3  | PReLU         | 1     
6  | model.convs.4  | MaxPool2d     | 0     
7  | model.convs.5  | Dropout       | 0     
8  | model.convs.6  | Conv2d        | 80 K  
9  | model.convs.7  | BatchNorm2d   | 80    
10 | model.convs.8  | PReLU         | 1     
11 | model.convs.9  | MaxPool2d     | 0     
12 | model.convs.10 | Dropout       | 0     
13 | model.convs.11 | Conv2d        | 80 K  
14 | mod

Val-Acc=0.01956135151155898


Val-Acc=0.6639004149377593
Train-Acc=0.7160079051383399


Val-Acc=0.5411973918197984


[I 2020-07-20 15:21:50,185] Setting status of trial#56 as TrialState.PRUNED. Trial was pruned at epoch 1.
GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.

   | Name           | Type          | Params
---------------------------------------------
0  | model          | AblationModel | 220 K 
1  | model.convs    | Sequential    | 162 K 
2  | model.convs.0  | BatchNorm2d   | 2     
3  | model.convs.1  | Conv2d        | 2 K   
4  | model.convs.2  | BatchNorm2d   | 40    
5  | model.convs.3  | PReLU         | 1     
6  | model.convs.4  | MaxPool2d     | 0     
7  | model.convs.5  | Dropout       | 0     
8  | model.convs.6  | Conv2d        | 80 K  
9  | model.convs.7  | BatchNorm2d   | 80    
10 | model.convs.8  | PReLU         | 1     
11 | model.convs.9  | MaxPool2d     |

...

Val-Acc=0.7664493183165383
Train-Acc=0.8458498023715415


Val-Acc=0.7889745109662122
Train-Acc=0.8452569169960474


Val-Acc=0.7783046828689982
Train-Acc=0.8460474308300395


Val-Acc=0.7972732661529343
Train-Acc=0.8496047430830039


Val-Acc=0.7759336099585062
Train-Acc=0.8480237154150198


In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
best_trial = study.best_trial
print("  Value: {}".format(best_trial.value))
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))